# Developing Activity class
__Keith Cheveralls__<br>
__March 2019__



In [ ]:
import os
import re
import sys
import git
import gzip
import time
import pickle
import fitparse
import datetime
import psycopg2
import subprocess
import sqlalchemy
import numpy as np
import pandas as pd

from scipy import stats
from psycopg2 import sql
from fitparse import FitFile
from matplotlib import pyplot as plt

In [ ]:
%matplotlib 
%load_ext autoreload
%autoreload 2

In [ ]:
sys.path.insert(0, '../../pgutils/')
import pgutils

sys.path.insert(0, '../')
import cypy2

root = '/home/keith/Downloads/export_7989839-1'
wahoo_example = '2326365683.fit.gz'
garmin_example = '2122584483.fit.gz'
garmin_indoor_example = '2324139976.fit.gz'

### Load a single activity directly from a FIT file

In [ ]:
a = cypy2.LocalActivity.from_fit_file(os.path.join(root, 'activities', garmin_example))

### Load all activities from a cached strava export

In [ ]:
strava_export = cypy2.StravaExportManager(root, from_cache=True)

In [ ]:
m = cypy2.ActivityManager.from_strava_export(strava_export)

In [ ]:
# testing id_from_fit
cypy2.LocalActivity.id_from_fit(file_id=pd.Series(dict(time_created=pd.to_datetime('2018-01-02 3:45:56'))))

### Sanity checks

In [ ]:
len(m.activities()), m.metadata().shape

In [ ]:
# check activity types
m.metadata().activity_type.unique()

In [ ]:
# count sensor flags by device
md = m.metadata()
pd.concat((md.groupby('device_model').count().activity_id, md.groupby('device_model').sum()), axis=1)

In [ ]:
# rides with power but without heart rate
m.metadata(activity_type='ride', heart_rate_flag=False, power_flag=True)

In [ ]:
# rides with heart rate but without an antplus_device_type column in device_info
# these *should* be only runs from fenix3
activities = []
for a in m.activities(activity_type='run'):
    if 'antplus_device_type' not in a._fit_data['device_info'].columns:
        if 'heart_rate' in a._fit_data['record'].columns:
            activities.append(a)
        
set([a.metadata.device_model for a in activities])

In [ ]:
# rides with cadence but no power - should all be from fr220
activities = []
for a in m.activities(activity_type='ride'):
    columns = a._fit_data['record'].columns
    if ('cadence' in columns) and ('power' not in columns):
        activities.append(a)
        
set([a.metadata.device_model for a in activities])

In [ ]:
m.activities()[-1]._fit_data['event']

In [ ]:
# cat all messages for some activities
dcat = pd.concat(tuple([a._fit_data['session'] for a in m.activities(activity_type='ride', device_model='edge520')]))

In [ ]:
dcat = dcat.dropna(axis='columns', how='all')

In [ ]:
dcat.tail()

### Populating the database

In [ ]:
user = 'keith'
host = 'localhost'
dbname = 'cypy2'
conn = psycopg2.connect(user=user, host=host, dbname=dbname)
cursor = conn.cursor()

engine = sqlalchemy.create_engine('postgresql://%s@%s/%s' % (user, host, dbname))

In [ ]:
pgutils.get_table_sizes(conn)

In [ ]:
for table in pgutils.get_table_names(conn):
    print('%s: %s' % (table, pgutils.get_column_names(conn, table)))

In [ ]:
# truncate all tables
for table in pgutils.get_table_names(conn):
    pgutils.truncate_table(conn, table, freal=True)

In [ ]:
a = m.activities()[-1]
a.to_db(conn)

In [ ]:
# attempt to insert all activities
start = time.time()

for activity in m.activities():
    sys.stdout.write('\r%s' % activity.metadata.activity_id)
    activity.to_db(conn)

print('\nElapsed time: %0.2f' % (time.time() - start))

### Loading all activities from the database

In [ ]:
user = 'keith'
host = 'localhost'
dbname = 'cypy2'
conn = psycopg2.connect(user=user, host=host, dbname=dbname)

In [ ]:
m = cypy2.ActivityManager.from_db(conn)

In [ ]:
m.metadata().shape

In [ ]:
a = m.activities()[-1]

### Insert processed data from all activities

In [ ]:
user = 'keith'
host = 'localhost'
dbname = 'cypy2_2'
conn = psycopg2.connect(user=user, host=host, dbname=dbname)

In [ ]:
a = m.activities()[-1]
pgutils.insert_value(conn, 'metadata', {'activity_id': a.metadata.activity_id})

In [ ]:
a.to_db(conn)

In [ ]:
# insert processed data from all activities
start = time.time()

for activity in m.activities():
    sys.stdout.write('\r%s' % activity.metadata.activity_id)
    activity.to_db(conn)

print('\nElapsed time: %0.2f' % (time.time() - start))

### Database debugging

In [ ]:
user = 'keith'
host = 'localhost'
dbname = 'cypy2_test'
conn = psycopg2.connect(user=user, host=host, dbname=dbname)

In [ ]:
selector={'activity_id': '20191122010203', 'date_created': '2019-03-12 22:23:53.694945-07'}

In [ ]:
kwargs = {'table': sql.Identifier('proc_records')}
kwargs['column'] = sql.SQL('*')

cols, vals = list(selector.keys()), list(selector.values())
kwargs.update({
    'filter_columns': sql.SQL(', ').join([sql.Identifier(col) for col in cols]),
    'filter_values': sql.SQL(', ').join([sql.Literal(val) for val in vals])
})
query = sql.SQL('select {column} from {table} where ({filter_columns}) = (%s, %s)').format(**kwargs)
query.as_string(conn)

In [ ]:
pgutils.execute_query(conn, query, vals)

In [ ]:
pgutils.insert_value(conn, 'metadata', {'activity_id': 'testid'})

In [ ]:
pgutils.insert_value(conn, 'proc_records', {'activity_id': 'testid', 'commit_hash': 'testhash'})

In [ ]:
d = pd.read_sql('select activity_id, date_created from proc_records', conn)
d.sort_values(by='date_created', ascending=False).iloc[0]

In [ ]:
conn.commit()

In [ ]:
conn.rollback()

In [ ]:
# all activity_ids
pgutils.get_rows(conn, 'metadata', column='activity_id').values.flatten()

In [ ]:
# events for one activity
pgutils.get_rows(conn, 'raw_events', selector={'activity_id': a.metadata.activity_id})

In [ ]:
# retrieve the timepoints as a dataframe
d = pgutils.get_rows(conn, 'raw_records', selector={'activity_id', a.metadata.activity_id})
pd.DataFrame(d.to_dict(orient='records').pop())

In [ ]:
# query debugging
query = sql.SQL('select {column} from {table}').format(**{
    'table': sql.Identifier('tablename'),
    'column': sql.SQL('*'),
})

query.as_string(conn)

### Analyzing timepoints

In [ ]:
# cat all timepoints from one column
d = pd.read_sql('select heart_rate from raw_records where heart_rate is not null', conn)
vals = np.concatenate(tuple(d.heart_rate.values))

In [ ]:
print('num nan/num total: %s/%s' % (np.isnan(vals).sum(), len(vals)))

vals[pd.isna(vals)] = np.nan
vals = vals.astype(float)

In [ ]:
# for heart rate
_ = plt.hist(vals[~np.isnan(vals)], bins=np.arange(60, 190, 2))

In [ ]:
# for power
_ = plt.hist(vals[~np.isnan(vals) & (vals < 500) & (vals > 0)], bins=np.arange(0, 500, 2))